<a href="https://colab.research.google.com/github/singwang-cn/Neural-Network/blob/master/ResNet_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
from torch.nn import init
from torch.nn import functional as F
import torchvision
import torchvision.transforms as T
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torchvision.datasets as dset
import numpy as np
import matplotlib.pyplot as plt

In [15]:
class ResBlock(nn.Module):
  def __init__(self, inp_channels, num_channels, use_1x1conv=False, strides=1) -> None:
    super().__init__()

    self.conv1 = nn.Conv2d (in_channels=inp_channels, out_channels=num_channels, kernel_size=3, stride=strides, padding=1)
    self.conv2 = nn.Conv2d (in_channels=num_channels, out_channels=num_channels, kernel_size=3, stride=1, padding=1)
    self.conv3 = None
    if use_1x1conv:
      self.conv3 = nn.Conv2d (in_channels=inp_channels, out_channels=num_channels, kernel_size=1, stride=strides)

    self.bn1 = nn.BatchNorm2d(num_channels)
    self.bn2 = nn.BatchNorm2d(num_channels)
    self.relu = nn.ReLU(inplace=True)

  def forward(self, X):
    Y = self.conv1(X)
    Y = self.bn1(Y)
    Y = F.relu(Y)

    Y = self.conv2(Y)
    Y = self.bn2(Y)

    if self.conv3:
      X = self.conv3(X)
    Y += X
    Y = F.relu(Y)
    return Y


In [29]:
def generate_blocks(input_channels, num_channels, num_blocks, first_block=False):
  blocks = []
  for i in range(num_blocks):
    if i == 0 and not first_block:
      blocks.append(ResBlock(input_channels, num_channels, use_1x1conv=True, strides=2))
    else:
      blocks.append(ResBlock(num_channels, num_channels))
  return blocks


In [32]:
from torch.nn.modules.linear import Linear
from torch.nn.modules.flatten import Flatten
def build_34layer_resnet(num_classes=1000):
  block1 = nn.Sequential(
      nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=1),
      nn.BatchNorm2d(64),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
  )
  block2 = nn.Sequential(*generate_blocks(64, 64, 3, first_block=True))
  block3 = nn.Sequential(*generate_blocks(64, 128, 4))
  block4 = nn.Sequential(*generate_blocks(128, 256, 6))
  block5 = nn.Sequential(*generate_blocks(256, 512, 3))
  fc = nn.Sequential(
      nn.AdaptiveAvgPool2d((1, 1)),
      nn.Flatten(start_dim=1, end_dim=-1),
      nn.Linear(512, num_classes)
  )

  return nn.Sequential(block1, block2, block3, block4, block5, fc)


In [40]:
def start_train(model, loader_tr, loader_va, show_epoch=1, num_epochs=10):
  loss_func = nn.CrossEntropyLoss()
  optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-4)
  cost = []
  for epoch in range(num_epochs):
    for x, y in loader_tr:
      y_pred = model(x)
      loss = loss_func(y_pred, y)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      cost.append(loss.detach().numpy())

    if (epoch+1) % show_epoch == 0:
      incorrect = 0
      for x, y in loader_va:
        y_pred = model(x).detach().numpy().argmax(axis=1)
        for i in range(len(y)):
          if y[i] != y_pred[i]:
            incorrect += 1
      print('epoch:{}, acurracy:{:.3f}, error rate:{:.3f}'.format(epoch+1, 1-incorrect/10000, incorrect/10000))


In [45]:
cifar10_tr = dset.CIFAR10("./dataset/cifar10", train=True, download=True, transform=T.ToTensor())
cifar10_te = dset.CIFAR10("./dataset/cifar10", train=False, download=True, transform=T.ToTensor())

NUM_TRAIN = 50000
NUM_VALID = 10000

batch_size = 256

loader_tr = DataLoader(cifar10_tr, batch_size=batch_size, shuffle=True)
loader_va = DataLoader(cifar10_te, batch_size=batch_size, shuffle=True)

HTTPError: ignored

In [ ]:
model = build_34layer_resnet(num_classes=10)
start_train(model, loader_tr, loader_va, show_epoch=1, num_epochs=1)

In [ ]:
#https://pystyle.info/pytorch-resnet/